In [1]:
from objects import EnvironmentManager
from helpers import *

env = EnvironmentManager()
env.start_vehicles()
env.route.load_all_pax()

observation, reward, terminated, truncated, info = env.step(action=None)
while not terminated:
    action = get_action('RA')
    observation, reward, terminated, truncated, info = env.step(action=action)

history = env.get_history()
# for key in history:
#     history[key]['scenario'] = scenario
#     history[key]['episode'] = i
#     results[key].append(history[key])

In [35]:
from rl_env import FlexSimEnv
import numpy as np

In [36]:
rl_env  = FlexSimEnv()

In [46]:
rl_env.reset()

({'control_stop_idx': array([0], dtype=int32),
  'n_requests': array([1], dtype=int32),
  'headway': array([563.], dtype=float32),
  'schedule_deviation': array([-56.], dtype=float32)},
 {'lost_requests': 0,
  'off_schedule_trips': 0,
  'total_trips': 0,
  'time': 706.0,
  'veh_idx': 1,
  'direction': 'out'})

In [59]:
rl_env.step(action=1)

({'control_stop_idx': array([3], dtype=int32),
  'n_requests': array([1], dtype=int32),
  'headway': array([506.], dtype=float32),
  'schedule_deviation': array([-33.], dtype=float32)},
 0.0,
 0,
 0,
 {'lost_requests': 0,
  'off_schedule_trips': 0,
  'total_trips': 0,
  'time': 3877.0,
  'veh_idx': 1,
  'direction': 'in'})

In [2]:
np.random.seed(0)
for i in range(2):
    rl_env = FlexSimEnv()
    observation, info = rl_env.reset()
    # action = int(action[0])
    observation, reward, terminated, truncated, info = rl_env.step(action=0)
    while not terminated:
        # Use the loaded agent to predict the action based on current observation
        observation, reward, terminated, truncated, info = rl_env.step(action=0)

NameError: name 'np' is not defined

In [4]:
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.env.env_context import EnvContext
from gymnasium import spaces
import numpy as np
import random

# --- Minimal multi-agent env with shared policy ---
class SharedExperienceEnv(MultiAgentEnv):
    def __init__(self, config: EnvContext):
        self._num_agents = config.get("num_agents", 3)
        self.agents = [f"agent_{i}" for i in range(self._num_agents)]
        self.observation_spaces = {
            agent: spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
            for agent in self.agents
        }
        self.action_spaces = {
            agent: spaces.Discrete(2)
            for agent in self.agents
        }
        self.state_memory = {agent: np.zeros(4) for agent in self.agents}
        self.current_agent = self.agents[0]

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        for agent in self.agents:
            self.state_memory[agent] = np.random.randn(4)
        self.current_agent = random.choice(self.agents)
        return {self.current_agent: self.state_memory[self.current_agent]}

    def step(self, action_dict):
        agent_id = list(action_dict.keys())[0]
        new_obs = np.random.randn(4)
        reward = np.random.rand()
        done = np.random.rand() < 0.05
        self.state_memory[agent_id] = new_obs
        next_agent = random.choice(self.agents)
        self.current_agent = next_agent
        return {agent_id: (new_obs, reward, done, {})}, {}, {}

# --- Init Ray ---
ray.init(ignore_reinit_error=True)

from ray.rllib.algorithms.ppo import PPOConfig

dummy_env = SharedExperienceEnv(EnvContext({"num_agents": 3}, worker_index=0))
example_agent = dummy_env.agents[0]

config = (
    PPOConfig()
    .environment(env=SharedExperienceEnv, env_config={"num_agents": 3})
    .multi_agent(
        policies={
            "shared_policy": (
                None,
                dummy_env.observation_spaces[example_agent],
                dummy_env.action_spaces[example_agent],
                {},
            )
        },
        policy_mapping_fn=lambda agent_id: "shared_policy",
    )
    .framework("torch")
)

trainer = config.build()
results = trainer.train()
print("✅ Mean reward:", results["episode_reward_mean"])

2025-04-02 10:56:30,402	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.
/Users/joseph/miniconda3/envs/rl-bus/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:512: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/joseph/miniconda3/envs/rl-bus/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/joseph/mini

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffec54a8b723e1fca4162c156601000000 Worker ID: ddf794f50831a6183d15d69613fdc1eb8aa96de06c9e7a587c9a51ec Node ID: 80baf190d189117b0a25003c668675e497da4d5ef53d4bc150a04d9d Worker IP address: 127.0.0.1 Worker port: 50908 Worker PID: 4320 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


2025-04-02 10:57:34,143	ERROR actor_manager.py:873 -- Ray error (The actor ec54a8b723e1fca4162c156601000000 is unavailable: The actor is temporarily unavailable: IOError: The actor is restarting.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
2025-04-02 10:57:34,144	ERROR actor_manager.py:873 -- Ray error (The actor 194f37903e54703cc23875af01000000 is unavailable: The actor is temporarily unavailable: IOError: The actor is restarting.. The task may or maynot have been executed on the actor.), taking actor 2 out of service.
2025-04-02 10:57:34,144	ERROR actor_manager.py:674 -- The actor ec54a8b723e1fca4162c156601000000 is unavailable: The actor is temporarily unavailable: IOError: The actor is restarting.. The task may or maynot have been executed on the actor.
NoneType: None
2025-04-02 10:57:34,144	ERROR actor_manager.py:674 -- The actor 194f37903e54703cc23875af01000000 is unavailable: The actor is temporarily unavailable: IOError: The actor i

KeyError: 'episode_reward_mean'

(MultiAgentEnvRunner pid=4536) 2025-04-02 10:57:34,451	WARNING env.py:37 -- Your MultiAgentEnv <SharedExperienceEnv<rllib-multi-agent-env-v0>> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(MultiAgentEnvRunner pid=4536) 2025-04-02 10:57:34,591	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
